## Connect to Meilisearch

In [1]:
import meilisearch

meilisearch_client = meilisearch.Client("http://localhost:7700", "aStrongMasterKey")
print("Successfully connected to Meilisearch!")

Successfully connected to Meilisearch!


## Get all documents

In [ ]:
index = meilisearch_client.index("articles")

all_documents = []
limit = 1000
offset = 0

print(f"Fetching all documents from the '{index.uid}' index...")

while True:
    response = index.get_documents({"limit": limit, "offset": offset})
    batch = response.results
    all_documents.extend(batch)

    if len(batch) < limit:
        break

    offset += limit

print(f"Total documents retrieved: {len(all_documents)}")

Fetching all documents from the 'articles' index...
Total documents retrieved: 10


## Reset the fields for each document

In [7]:
documents_to_update = []

for document in all_documents:
    documents_to_update.append(
        {"id": document.id, "view_count": 0, "claps_count": 0, "read_count": 0}
    )

Now, let's update the documents in MeiliSearch.

In [10]:
task = index.update_documents(documents_to_update)
meilisearch_client.wait_for_task(task.task_uid)

Task(uid=1014, index_uid='articles', status='succeeded', type='documentAdditionOrUpdate', details={'receivedDocuments': 10, 'indexedDocuments': 10}, error=None, canceled_by=None, duration='PT0.279961149S', enqueued_at=datetime.datetime(2025, 11, 2, 20, 33, 36, 258357), started_at=datetime.datetime(2025, 11, 2, 20, 33, 36, 263818), finished_at=datetime.datetime(2025, 11, 2, 20, 33, 36, 543779))

Let's verify that the documents have been updated.

In [11]:
offset = 0
new_all_documents = []

print(f"Fetching all documents from the '{index.uid}' index...")

while True:
    response = index.get_documents({"limit": limit, "offset": offset})
    batch = response.results
    new_all_documents.extend(batch)

    if len(batch) < limit:
        break

    offset += limit

print(f"Total documents retrieved: {len(new_all_documents)}")

Fetching all documents from the 'articles' index...
Total documents retrieved: 10


In [12]:
for document in new_all_documents:
    assert document.view_count == 0
    assert document.claps_count == 0
    assert document.read_count == 0

No errors that means the fields have been successfully reset!